# Easy Manipulation of Genetic Variant Data

For analysis of genetic variants, single nucleotide polymorphism (SNP) information is widely used. 

- A SNP corresponds to a nucleotide position on the genome where some degree of variation has been observed in a population, with each individual have one of several possible alleles at that position on each of a pair of chromosomes. 
- The alleles are often classified as a reference allele (REF, allele on the [reference genome](https://en.wikipedia.org/wiki/Reference_genome)) and alternative alleles (ALT).  
- The most widely used repository for SNP information is [dbSNP](https://www.ncbi.nlm.nih.gov/snp/), where each SNP is indexed by an identifier beginning with `rs` ("rsID"). 
- We commonly use biallelic SNPs for many analyses. We will assume biallelic SNPs for this tutorial.


The _genotypes_ of each sample for each variant is commonly coded in:

| value | genotype |
|:---:|:---:|
| 0 | homozygous REF |
| 1 | heterozyguous REF/ALT |
| 2 | homozygous ALT |

Sometimes, when the genotype has uncertainties, it is represented in _dosage_ after imputation. Given the posterior probabilities of each genotype, dosage is computed as 
$$\mathrm{dosage = 0 \cdot Prob(REF/REF) + 1 \cdot Prob(REF/ALT) + 2 \cdot Prob(ALT/ALT)}$$
and have values in $[0, 2]$. 



Often, the data formats for genetic variants include
- Information of $m$ samples (identifier, sex, phenotypes, etc.)
- Information of $n$ SNPs (identifier, chromosome, position, REF/ALT alleles, etc.)
- An $m \times n$ table or matrix containing the observed allelic type at $n$  positions for $m$ individuals.

A common type of analysis for this data is [genome-wide association studies (GWAS)](https://en.wikipedia.org/wiki/Genome-wide_association_study), often testing a statistical hypothesis variant by variant. Significance of each SNP is assessed by some type of regression:
$$
\mathrm{trait ∼ SNP + age + sex +  principal\;components + other\;covariates }
$$

In this workshop, we learn four widely-used file types for genetic variants and how to manipulate them in Julia. We focus on accessing genotype information variant by varint, as it is a common workflow for a GWAS-based application. We learn also try to compute some simple properties of SNPs such as minor allele frequencies (MAF). 

__Why there are so many formats...__

Genetics is a fast-moving subject driven by academics. In the early stage, they devise format that best suit their needs, rather than using what is out there. As the field matures, it converges to a couple of dominant formats. The formats covered in this workshop are considered to be a relatively dominant ones. There are many, many more out there. 

## Genetic Variant File Formats


### Variant Call Format (`.vcf`)
Text-based format is the most intuitive to represent genetic variants. It is the most flexible format to include diverse information on the samples and variants.

(For raw text format)
- __Pros__: Highly flexible, easy to parse and interpret
- __Cons__: Large file size

With the scale of recent genetic data, with data set like UK Biobank having near a million subjects and millions of variants, the storage needed for storing a raw VCF file is prohibitively huge. A common approach to remedy this is storing the data in a compressed form (such as `.gz` file) and decompress it at analysis time as a stream. However, there is a trade-off between stored file size and time needed for decompression.

(For compressed format)
- __Pros__: Reasonable file size with flexible representation
- __Cons__: Takes too long to decompress





#### Basic usage: `VCFTools.jl`

In [ ]:
using VCFTools

fh = openvcf("test_vcf.vcf.gz", "r")
for l in 1:35
    println(readline(fh))
end
close(fh)

As in typical VCF files, it has a bunch of meta-information lines, one header line for column names, and then one line for each marker. In this VCF, genetic data has fields GT (genotype), DS (dosage), and GL (genotype likelihood).

To access number of records and samples (individuals):

In [ ]:
records = nrecords("test_vcf.vcf.gz")

In [ ]:
samples = nsamples("test_vcf.vcf.gz")

Information on samples and variants can be retrieved using the `VariantCallFormat` package:

Sample names can be retrieved by: 

In [ ]:
using VariantCallFormat
reader = VCF.Reader(openvcf("test_vcf.vcf.gz"))
h = header(reader)
h.sampleID

Information of each variant is accessible by:

In [ ]:
reader = VCF.Reader(openvcf("test_vcf.vcf.gz"))
println("chrom\tposition\tids\t\treference\talternative")
cnt = 0
for record in reader
    println("$(VCF.chrom(record))\t$(VCF.pos(record))\t$(try VCF.id(record) catch; ["."] end)\t$(VCF.ref(record))\t\t$(VCF.alt(record))")
    cnt += 1
    if cnt == 30
        break
    end
end

Each row of a VCF file represents a single variant, so it is natural to parse the genotypes or dosages variant-by-variant. The function `copy_gt!()` computes genotypes of each variant, and `copy_ds!()` computes dosages of each variant, represented by values in $[0, 2]$. 



In [ ]:
using VariantCallFormat
using Statistics
using StatsBase
# initialize VCF reader
people, snps = nsamples("test_vcf.vcf.gz"), nrecords("test_vcf.vcf.gz")
reader = VCF.Reader(openvcf("test_vcf.vcf.gz"))

# pre-allocate vector for marker data
g = zeros(Union{Missing, Float64}, people)
sampleID = Vector{String}(undef, people)
rec_chr = Vector{String}(undef, 1)
rec_pos = Vector{Any}(undef, 1)
rec_ids = Vector{Vector{String}}(undef, 1)
rec_ref = Vector{String}(undef, 1)
rec_alt = Vector{Vector{String}}(undef, 1)

for j = 1:30
    copy_gt!(g, reader; model = :additive, impute = true, 
        sampleID=sampleID,
        record_chr=rec_chr,
        record_pos=rec_pos,
        record_ids=rec_pos,
        record_ref=rec_ref,
        record_alt=rec_alt)
    # Do some statistical analysis. Just for simple illustration, we show the mean of the genotype values.
    println(mean(g))
end
close(reader)

The keyword arguments: 
- `model` defines which genotype model to use. The common choice is `:additive` for 0/1/2 encoding. 
- `impute` sets whether to impute the missing values with the mean of nonmissing values.
- `sampleID` stores the list of sample ID.
-`record_chr`, `record_pos`, `record_ids`, `record_ref`, and `record_ref` stores chromosome, position, identifiers, reference allele, and alternative alleles, respectively.  

### PLINK 1 BED format (`.bed` + `.bim` and `.fam`)

In order to rapidly process the variant information while keeping the file size small, various binary file types are devised. This format is basically a compact array of two-bit representation of genotypes. 
This is the native format of the well-celebrated large-scale GWAS tool, [PLINK 1](https://www.cog-genomics.org/plink/). 
This is suitable for high-performance applications directly dealing with genotype matrices such as [iterative hard thresholding](https://github.com/OpenMendel/MendelIHT.jl) and [admixture analysis](https://github.com/OpenMendel/OpenADMIXTURE.jl).  Major weakness of this format is that it cannot contain imputed information when there is uncertainty in genotypes.

| Genotype | Plink/SnpArray (hexadecimal byte) |  binary | numeric |
|:---:|:---:|:---:|:---:|
| A1,A1 | `0x00` | `0b00` | `0` |
| missing | `0x01` | `0b01` | `1` |
| A1,A2 | `0x02` | `0b10` | `2` |
| A2,A2 | `0x03` | `0b11` | `3` |


- __Pros__: 
    - Small size. 1/32, 1/16, 1/4 compared to double-precision/single-precision/single character representation
    - Fixed-width format suitable for massively accelerated computation (e.g. GPU acceleration)
    - Only takes $O(n)$ time to parse $n$ variants of a single sample
- __Cons__: 
    - Only contains hard-called genotypes (0, 1, 2, or missing), and cannot contain imputed values
    - Need external files for variant (`.bim`) and sample (`.fam`) information
    - REF/ALT designation unsupported
    
#### Basic usage: `SnpArrays.jl`

In [ ]:
using SnpArrays
d = SnpData("test_bed")

Information of i-th sample is accessible by:

In [ ]:
i = 20
d.person_info[i, :]

Information of j-th variant is accessible by:

In [ ]:
j = 7
d.snp_info[j, :]

Genotype of sample $i$, variant $j$ is accessible by:

In [ ]:
d.snparray[i, j]

Note that this is the encoding defined by the table above. If converted numerically in `0`, `1`, `2`-encoding, it corresponds to `2`. 

Genotypes of each variant in numeric form can be accessed by:

In [ ]:
g = Vector{Float64}(undef, d.people)
cnt = 0
for j in 1:30
    copyto!(g, @view(d.snparray[:, j]); impute=true, 
        model=ADDITIVE_MODEL, 
        center=false, scale=false)
    # Do some statistical analyses
    println(mean(g))
end

Note that the count of the second allele is encoded, and it is often the reference allele. If it is desired to run the analyses based on the alternative allele count, the values have to be reversed (subtracted from `2.0`). This has led to a lot of confusion, and some later projects began to put the reference allele first (most notably, UK Biobank). 

### Oxford BGEN format (`.bgen` + optional `.sample`)
The BGEN format is native to Oxford statistical genetics tools, such as [IMPUTE2](https://mathgen.stats.ox.ac.uk/impute/impute_v2.html) and [SNPTEST](https://www.well.ox.ac.uk/~gav/snptest/). This format employs variant-by-variant compression scheme, well-tailored for GWAS applications. The UK Biobank imputed data is distributed in this format. Compression algorithms supported are [Gzip](https://www.gnu.org/software/gzip/) (used in `.gz` files) and [Zstandard](https://facebook.github.io/zstd/) (used in `.zst` files)

- __Pros__: 
    - Very small file size. Further compression from the fixed-width genotype representation
    - Admits variable precision.
    - Variant-by-variant compression suitable for GWAS
    - Allows dosage and phase information
- __Cons__: 
    - Needs an external index for random access
    - Compression not tailored for genetic context, very slow to access genotypes of a single individual
    - REF/ALT designation unsupported
    
#### Basic usage (`BGEN.jl`)

In [ ]:
using BGEN
b = Bgen("test_bgen.bgen")

Sample names are accessible by: 

In [ ]:
BGEN.samples(b)

Since how to iterate across the variants in BGEN files depend on if the index file is provided, we support an iterator interface for variants. If you are familiar with Python, it is an interface similar to generator defined using `yield` statement in Python. This iterator is accessible by the function `iterator()`. 

In [ ]:
println("rsid\t\tchrom\tpos\t\t#alleles\tlist of alleles")
for (i, v) in enumerate(BGEN.iterator(b))
    println("$(rsid(v))\t$(chrom(v))\t$(pos(v))\t$(n_alleles(v))\t\t$(alleles(v))")
    if i == 30
        break
    end
end

Dosage of each variant is accssible by: 

In [ ]:
for (i, v) in enumerate(BGEN.iterator(b))
    g = first_allele_dosage!(b, v) # first allele is the ALT allele in this file.
    println(mean(g))
    if i == 30
        break
    end
end

### PLINK 2 PGEN format (`.pgen` + `.pvar` and `.psam`)

PGEN is a backward-compatible extension of the BED format for [PLINK 2](https://www.cog-genomics.org/plink/2.0/) under development. It tries to overcome the limitation of the BED format, and can incorporate phase and dosage information. Cutting-edge GWAS tools now support this format. 

- __Pros__: 
    - Very small file size, similar to BGEN.
    - Variant-by-variant compression suitable for GWAS. 
    - Allows dosage and phase information
    - Difflist or patch list: genetics-inspired compression incorporating linkage disequillibrium
        - 4x Faster than reading in `BGEN` with similar precision
- __Cons__: 
    - Still under development
    - Difficult to write a parser by admitting too many forms, including the whole BED format.
    - Requires external variant (`.pvar`) and sample (`.psam`) files. 
    
Note: [Linkage disequilibrium](https://www.sciencedirect.com/topics/neuroscience/linkage-disequilibrium)

> Linkage disequilibrium (LD) is the correlation between nearby variants such that the alleles at neighboring polymorphisms (observed on the same chromosome) are associated within a population more often than if they were unlinked.

#### Basic usage

In [ ]:
using PGENFiles

In [ ]:
p = Pgen("test_pgen.pgen");

In [ ]:
print(PGENFiles.n_variants(p)) 

In [ ]:
print(PGENFiles.n_samples(p))

In [ ]:
v_iter = PGENFiles.iterator(p);

In [ ]:
v = first(v_iter)
g, _... = get_genotypes(p, v)

`g` is the genotype vector.

In [ ]:
g

The encoding is as follows:

| genotype code |	genotype category |
|:---:|:---:|
| `0x00` | 	homozygous REF |
| `0x01` |	heterozygous REF-ALT |
| `0x02` |	homozygous ALT |
| `0x03` |	missing |

To avoid array allocations for iterative genotype extraction, one may preallocate `g` and reuse it.

In [ ]:
g = Vector{UInt8}(undef, PGENFiles.n_samples(p))
for (i, v) in enumerate(PGENFiles.iterator(p))
    get_genotypes!(g, p, v)
    println(mean(g))
    # do someting with genotypes in `g`...
    if i == 30
        break
    end
end

Similarly, ALT allele dosages are available through the functions `alt_allele_dosage()` and `alt_allele_dosage!()`. As genotype information is required to retrieve dosages, space for genotypes are also required for `alt_allele_dosage!()`. These functions return dosages, parsed genotypes, and endpoint of the dosage information in the current variant record.

In [ ]:
d = Vector{Float32}(undef, PGENFiles.n_samples(p))
g = Vector{UInt8}(undef, PGENFiles.n_samples(p))
g_ld = similar(g)
for (i, v) in enumerate(v_iter)
    alt_allele_dosage!(d, g, p, v)
    # do someting with dosage values in `d`...
    println(mean(d))
    if i == 30
        break
    end
end

Information of each sample and variant is available by reading in `.psam` and `.pvar` file as a `DataFrame` (not yet supported by `PGENFiles.jl`). `.pvar` format admits regular `.vcf` format.

In [ ]:
using DataFrames, CSV

In [ ]:
sample_info = CSV.read("test_pgen.psam", DataFrame)
first(sample_info, 5)

For `.pvar` file, all the lines starting with `#` is header, ending with the column names. For `test_pgen.pvar`, it is the 26th line. 

In [ ]:
variant_info = CSV.read("test_pgen.pvar", DataFrame; delim="\t", header=26)
first(variant_info, 5)

## Exercise

[Minor allele frequency](https://en.wikipedia.org/wiki/Minor_allele_frequency) (MAF) is widely used for determining if a variant is rare or frequent, and in GWAS, it is used for measuring information content of a variant. One such measure is the ratio of actual numerical variance and expected variance from the binomial model ($2 \hat{p}(1-\hat{p})$, where $\hat{p}$ is the MAF in $[0, 1]$). 

By modifying the code cells above...
1. Compute minor allele frequencies using the packages we used. 
2. Determine the minor/major allele.
3. Compute the variance ratio measure? 
4. Print out the minor allele, MAF, and the variance ratio for each variant.

## File format transformation

The file formats can be transformed between each other using plink2 on the command line. For example, the files used in this workshop is transformed from a VCF file using the following commands.

In [ ]:
run(`plink2 --vcf test_vcf.vcf.gz --export bgen-1.3 --out test_bgen`) # ALT allele comes first

In [ ]:
run(`plink2 --vcf test_vcf.vcf.gz --make-bed --out test_bed`) # ALT allele comes first

In [ ]:
run(`plink2 --vcf test_vcf.vcf.gz --make-pgen --out test_pgen`)

## Concluding Remarks: Applications in OpenMendel

While we focused on variant-by-variant access of the genetic variant data, many of the packages we have seen has various utility functionalities such as filtering out the variants with low MAF or low genotype success rate, filter by chromosome, and merge. In case of the BED format (`SnpArrays.jl`), we support high-performance linear algebra on the genotype matrix which supports multithreading and GPU (graphics programming unit) computation. 

### GWAS application

- `OrdinalGWAS.jl` : GWAS for ordered categorical trait, e.g. disease status (undiagnosed, pre-disease, mild, moderate, severe)
- `TrajGWAS.jl` : GWAS for continuous longitudinal phenotypes using a modified linear mixed effects model. Tests both mean effect and within-subject variability effect.


### High-performance applications using fixed-width BED format

- `MendelIHT.jl` : Rather than GWAS with variant-by-variant testing, uses penalized regression model. 
- `OpenADMIXTURE.jl` : Estimates ancestry of samples. Julia reimplementation of highly celebrated ADMIXTURE software in C++, 8x faster in multi-threaded setting and 35x faster using GPU.


### Others
- `VarianceComponentModels.jl` : Fitting and testing variance component models
- `MendelImpute.jl` : Fast phase inference and genotype imputation
- `TraitSimulation.jl` : Quickly simulate phenotypes under a variety of genetic architectures.
- `QuasiCopula.jl` : Analysis of correlated data with specified marginals with a flexible quasi-copula distribution